<a href="https://colab.research.google.com/github/totvslabs/carol-notebooks/blob/main/notebooks/GetDataBigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Carol Get Data from BQ
`This script will query a datamodel in BQ and will return the result in a dataframe.`
`The script will ask for the following json on execution:`

```python
{
    "authentication_config" : {
        "username": "username@totvs.com.br",
        "password": "password",
        "organization": "YourOrganization",
        "tenantName": "YourTenantName"
    },
    "script_config" : {
        "datamodels": [
          "datamodel1",
          "datamodel2",
          "datamodel3"
        ]
    }
}
```
`You need TENANT ADMIN permissions to run this script.`

#### REQUIREMENTS
`These are the packages the script needs before execution.`

In [ ]:
%%capture
!pip install --quiet "pycarol==2.54.9" 
from google.colab import files
import json, os, sys
from google.api_core import exceptions
from google.cloud import bigquery
from google.oauth2.service_account import Credentials
import pycarol

#### CAROL LOGIN FUNCTIONS
`These are the functions made to login into Carol. They will be the same for all notebooks (ideally) and will use pyCarol.` 

[pyCarol reference](https://github.com/totvslabs/pyCarol)

In [ ]:
def carol_connect(username, password, organization, tenantName):
    print(f"Connecting to Carol tenant {tenantName}... ", end="\n")

    return pycarol.Carol(domain=tenantName,
                auth=pycarol.PwdAuth(username, password), organization=organization)

#### SCRIPT FUNCTIONS
`If the script requires more functions to execute, they will be here.`

In [ ]:
def get_data_bq(carolObject, datamodel):
    credentials = pycarol.bigquery.BQ(Carol).get_credential()

    environment = Carol.get_current()["env_id"]
    project=f"carol-{environment[0:20]}"
    dataset=f"{project}.{environment}"

    service_account = Credentials.from_service_account_info(credentials['service_account'])
    bq = bigquery.Client(project=project, credentials=service_account)
    config = bigquery.QueryJobConfig(priority="BATCH", default_dataset=dataset)

    sql = f"""
    SELECT * EXCEPT(mdmCounterForEntity__DATETIME__,mdmCounterForEntity,mdmStagingCounter,mdmId,mdmCreated,mdmLastUpdated,mdmTenantId,mdmEntityType,mdmSourceEntityNames,mdmCrosswalk,mdmStagingRecord,mdmApplicationIdMasterRecordId,mdmPreviousIds,mdmDeleted),
    FROM `{dataset}`.{datamodel}
    """

    try:
        result = (
            bq.query(sql, config)
            .result()
            .to_dataframe(
                create_bqstorage_client=True,
            )
        )
        result.to_csv(f'./{datamodel}.csv', index=False)
        print(f'result of table `{dataset}`.{datamodel} extracted to ./{datamodel}.csv')
        return(result)

    except exceptions.ClientError:
        print({"error" : str(sys.exc_info()[1])})
        return({"error" : str(sys.exc_info()[1])})

#### CONFIGURATION FILE
`Now you will need to upload the configuration file with the format given above.`

In [ ]:
try:
    config_file = files.upload()
    config_json = json.loads(config_file[next(iter(config_file))].decode("utf-8"))
except:
    with open('./carol.json') as config_file:
        config_json = json.loads(config_file.read())
    config_file.close()

#### SCRIPT EXECUTION
`The main execution of the script will happen here.`

In [ ]:
Carol = carol_connect(
    config_json['authentication_config']['username'], 
    config_json['authentication_config']['password'], 
    config_json['authentication_config']['organization'], 
    config_json['authentication_config']['tenantName'])

print('If you are using Google Colaboratory, remember to refresh the contents of working folder after script execution!')

for datamodel in config_json['script_config']['datamodels']:
    get_data_bq(Carol, datamodel)